In [3]:
#Setting up the notebook.
import pandas as pd
# I'm working with some data sets with a lot of columns, so I added this command to reveal all of them.
pd.set_option('display.max_columns', None)

In [4]:
#Here I imported the data and assigned it names.
nyc_chargers = pd.read_csv('../data/NYC EV chargers.csv')
nyc_cars = pd.read_csv('../data/NY Vehicle registrations.csv')
boston_chargers = pd.read_csv('../data/Boston Electric_Vehicle_Charging_Stations.csv')
boston_cars = pd.read_csv('../data/Boston_Bi_Annual.csv')

In [5]:
#Here I am checking for the formatting for the columns just to ensure I'm writing the next line peroperly.
nyc_cars.head()

,Record Type,VIN,Registration Class,City,State,Zip,County,Model Year,Make,Body Type,Fuel Type,Unladen Weight,Maximum Gross Weight,Passengers,Reg Valid Date,Reg Expiration Date,Color,Scofflaw Indicator,Suspension Indicator,Revocation Indicator
0,BOAT,999999999999,BOT,HILTON,NY,14468,MONROE,1975.0,STARC,BOAT,GAS,NaN,NaN,NaN,08/24/2022,07/31/2025,NaN,N,N,N
1,VEH,9999236,HIS,ROSLYN,NY,11576,NASSAU,1937.0,CHRY,4DSD,GAS,6300.0,NaN,NaN,05/07/2024,06/23/2025,BK,N,N,N
2,TRL,9999,LTR,BETHEL PARK,PA,15102,OUT-OF-STATE,1980.0,CRCF,LTRL,NONE,NaN,3000.0,NaN,11/15/2023,12/31/2024,BR,N,N,N
3,BOAT,999747,BOT,CORTLAND,NY,13045,CORTLAND,1971.0,STARC,BOAT,GAS,NaN,NaN,NaN,03/08/2023,04/30/2026,NaN,N,N,N
4,BOAT,999733,BOT,CANEADEA,NY,14717,ALLEGANY,1971.0,STARC,BOAT,GAS,NaN,NaN,NaN,07/25/2024,06/30/2027,NaN,N,N,N


In [6]:
#This line is the first filtering the New York State Vehicle Registration data.
nyc_evs = nyc_cars[
    #This line filtered out other vehicle types like boats and snow mobiles.
    (nyc_cars['Record Type'] == 'VEH') &
    #This filtered the vehicles down to passenger vehicles, vutting out things like semis, box trucks, etc.
    (nyc_cars['Registration Class'] == 'PAS') &
    #This line filtered the entries by county of vehicle use to to NYC. 
    (nyc_cars['County'].isin(['NEW YORK', 'KINGS', 'QUEENS', 'BRONX', 'RICHMOND'])) &
    #This line filtered the registrations by battery eletric.
    (nyc_cars['Fuel Type'] == 'ELECTRIC')
]

In [7]:
#This was checking the filtering.
len(nyc_evs)

24664

In [8]:
#Here I was creating a total number of EVs to be used later for figuring out cars per plug.
total_nyc_evs = len(nyc_evs)

In [9]:
#Here I wanted to confirm the charger name
nyc_chargers['TYPE OF CHARGER'].unique()

array(['Level 3 Fast Charger', 'Level 2 Charger', 'EV Solar Arc Charger',
       'Level 3 Fast Charger - Boost', 'EV Solar Canopy Charger',
       'L2 DOT Flo Curbside Charger', 'DOT Municipal Level 2 Charger',
       'DOT Municipal Level 3 Charger', 'Mobile Charger'], dtype=object)

In [10]:
#Here I filtered the chargers down to the ones avaliable to the public.
nyc_public_chargers = nyc_chargers[(nyc_chargers['TYPE OF CHARGER'] == 'L2 DOT Flo Curbside Charger')]

In [11]:
#Here I created a total of chargers by filtering the data set down to those only avaliable to the public.
nyc_public_chargers_total_plugs = nyc_public_chargers['NO. OF PLUGS'].sum()

In [12]:
#This step had me filtering down the Massachusetts Motor Vehicle Census to determine how many EVs were on roads as of 7/1/24
boston_evs = boston_cars[
    #This condition filtered the data down to the most recent check.
    (boston_cars['Date'] == '2024-07-01') &
    #This condition filtered the location down to Boston specifically.
    (boston_cars['Municipality'] == 'BOSTON') &
    #This condition limited the type of vehicle down to passenger cars.
    (boston_cars['VehicleUse'] == 'Passenger') &
    #This condition set the type to electric vehicle.
    (boston_cars['AdvancedVehicleType'] == 'Electric Vehicle')
]

In [13]:
#Because the MVC works differently than the NYC registration data, I had to sum the vehicle counts from that to be able to get the number of EVs registered in the region.
total_boston_evs = boston_evs['Count'].sum()

In [14]:
#Here I filtered down the chargers in Boston from the dataset to fit the same requirements as the NYC data set.
boston_public_chargers = boston_chargers[
    #This condition had me filtering down to the chargers provided by the state, local and federal government in the area.
    (boston_chargers['owner_type_code'].isin(['LG', 'SG', 'FG'])) &
    #This condition limited the area to Boston.
    (boston_chargers['city'] == 'Boston')
].copy()

In [15]:
#This is establishing the number of vehicles to chargers for NYC by dividing the cars by the chargers.
nyc_vehicles_per_charger = total_nyc_evs / nyc_public_chargers_total_plugs

In [16]:
#print(boston_vehicles_per_charger)
print(nyc_vehicles_per_charger)

209.01694915254237


In [17]:
#This is establishing the number of vehicles to chargers for Boston by dividing the cars by the chargers.
#(I had to just hand count them, something broke in the code when I tried to sum them that way and I wanted to make deadline!)
boston_vehicles_per_charger = total_boston_evs / 20

In [18]:
#This is showing off the ratio for Boston.
print(boston_vehicles_per_charger)

360.9


In [19]:
print(total_boston_evs)

7218
